In [64]:
#System imports
from __future__ import print_function
from __future__ import division
import os
import glob
import json
import collections
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [94]:
#Extra import
%matplotlib inline
import numpy as np
import SimpleITK as sitk
from joblib import Parallel, delayed
import nibabel as nib
from matplotlib import pyplot as plt

In [10]:
datapath = os.path.join(os.path.dirname(os.getcwd()), "BRATS_2016")

In [145]:
impaths = [path for path in glob.glob(os.path.join(datapath, "sourcedata", "*", "*", "*", "*")) if "ce-GADOLINIUM_T1w.nii" in path]
impath = impaths[40]

In [113]:
def show_im2(x, y):
    plt.imshow(x[:,:,y].T, cmap="gray")
    plt.axis('off')
    

In [146]:
im = nib.load(impath)
im_arr = im.get_data()
print("Image dimension:", im.shape)
print("Voxel size (mm):", im.header.get_zooms())
sx, sy, sz = im.shape

def show_axial(im_arr, mr_slice):
    plt.imshow(im_arr[:,:,mr_slice].T, cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

interactive_plot = interactive(show_axial, im_arr=fixed(im_arr), mr_slice=(0, sz, 1))
interactive_plot    


Image dimension: (240, 240, 155)
Voxel size (mm): (1.0, 1.0, 1.0)


interactive(children=(IntSlider(value=77, description='mr_slice', max=155), Output()), _dom_classes=('widget-i…

In [133]:
def show_sagittal(im_arr, mr_slice):
    plt.imshow(np.flipud(im_arr[mr_slice, :, :].T), cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

interactive_plot = interactive(show_sagittal, im_arr=fixed(im_arr), mr_slice=(0, sx, 1))
interactive_plot   

interactive(children=(IntSlider(value=120, description='mr_slice', max=240), Output()), _dom_classes=('widget-…

In [135]:
def show_coronal(im_arr, mr_slice):
    plt.imshow(np.flipud(im_arr[:, mr_slice, :].T), cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

interactive_plot = interactive(show_coronal, im_arr=fixed(im_arr), mr_slice=(0, sy, 1))
interactive_plot   

interactive(children=(IntSlider(value=120, description='mr_slice', max=240), Output()), _dom_classes=('widget-…